In [1]:

# Check Python & GPU
import sys, torch
print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())



Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
CUDA available: True


In [3]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [1]:
!pip install ultralytics opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()  # choose your Roboflow dataset zip

Saving Helmet Detection_YOLOv8.v3-with-and-without-helmet-dataset.yolov8.zip to Helmet Detection_YOLOv8.v3-with-and-without-helmet-dataset.yolov8.zip


In [6]:
!unzip "Helmet Detection_YOLOv8.v3-with-and-without-helmet-dataset.yolov8.zip" -d helmet_dataset

Streaming output truncated to the last 5000 lines.
 extracting: helmet_dataset/train/images/train567_jpg.rf.9f629a5ca8fe354a18b66ac2c52e3073.jpg  
 extracting: helmet_dataset/train/images/train568_jpg.rf.4f5aef94083a1dcc36a1e7173f12ace8.jpg  
 extracting: helmet_dataset/train/images/train568_jpg.rf.7258819660acc7f8b7216edf210c67b4.jpg  
 extracting: helmet_dataset/train/images/train568_jpg.rf.e84c9baa6aae8da4fe66beec337c708c.jpg  
 extracting: helmet_dataset/train/images/train569_jpg.rf.244fc2f08958b8fcbe5f4535be2b279c.jpg  
 extracting: helmet_dataset/train/images/train569_jpg.rf.c1345214ec730bdb9a3e62fc5634d5bd.jpg  
 extracting: helmet_dataset/train/images/train569_jpg.rf.cf1867a0c144ac9f62c520203a6549cb.jpg  
 extracting: helmet_dataset/train/images/train570_jpg.rf.5d2c30252d7eff982494c2e04ba14758.jpg  
 extracting: helmet_dataset/train/images/train570_jpg.rf.abe53a90a94886efac271ce7ff70a646.jpg  
 extracting: helmet_dataset/train/images/train570_jpg.rf.cb9d4b2366d1b6e976df5ee4284a

In [7]:
!ls helmet_dataset

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [9]:
import torch
from ultralytics import YOLO

# Load YOLOv8 nano model (fastest for demo)
model = YOLO("yolov8n.pt")

results = model.train(
    data="helmet_dataset/data.yaml",  # path to dataset config
    epochs=10,                        # small number for sprint
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else "cpu"
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=helmet_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

In [11]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Load trained model
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Test on an uploaded video
cap = cv2.VideoCapture("helmet_test.mp4")  # replace with your video file
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    annotated_frame = results[0].plot()
    cv2_imshow(annotated_frame)  # show inline in Colab
    break  # remove break to process full video
cap.release()

In [12]:
from google.colab import files

# Download the best model weights
files.download("/content/runs/detect/train/weights/best.pt")

# (Optional) also download the last checkpoint
# files.download("/content/runs/detect/train/weights/last.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!ls



sample_data
